In [34]:
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
from requests import get
h2o.init()
if not H2ODeepWaterEstimator.available(): exit

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /home/ubuntu/h2o-3/build/h2o.jar
  Ice root: /tmp/tmpVM1Z0Q
  JVM stdout: /tmp/tmpVM1Z0Q/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpVM1Z0Q/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,37 minutes
H2O cluster name:,H2O_from_python_ubuntu_a2mxf2
H2O cluster total nodes:,1
H2O cluster free memory:,3.264 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [35]:
print("Downloading the model")
## http://data.dmlc.ml/mxnet/models/imagenet/inception-bn_old.tar.gz
## !gunzip ...

print("Importing the model architecture for scoring in H2O")
model = H2ODeepWaterEstimator(epochs=0, ## no training - just load the state - NOTE: training for this 3-class problem wouldn't work since the model has 1k classes
                            mini_batch_size=32, ## mini-batch size is used for scoring
                            ## all parameters below are needed
                            network='user',
                            network_definition_file=os.getcwd() + "/Inception_BN-symbol.json",
                            network_parameters_file=os.getcwd() + "/Inception_BN-0039.params",
                            mean_image_file=        os.getcwd() + "/mean_224.nd",
                            image_shape=[224,224],
                            channels=3
)

frame = h2o.import_file("/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat_dog_mouse.csv")
print(frame.head(5))
nclasses = frame[1].nlevels()[0]

model.train(x=[0],y=1, training_frame=frame) ## must call train() to initialize the model, but it isn't training

Importing the model architecture for scoring in H2O
Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2
/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat/102194502_49f003abd9.jpg,cat
/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat/11146807_00a5f35255.jpg,cat
/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat/1140846215_70e326f868.jpg,cat
/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat/114170569_6cbdf4bbdb.jpg,cat
/home/ubuntu/h2o-3/bigdata/laptop/deepwater/imagenet/cat/1217664848_de4c7fc296.jpg,cat



deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [36]:
## Extract deep features from final layer before going into Softmax.
extracted_features = model.deepfeatures(frame, "global_pool_output")
#extracted_features2 = model.deepfeatures(frame, "conv_5b_double_3x3_1_output")

print(extracted_features)
assert extracted_features.ncol == 1024


deepfeatures progress: |██████████████████████████████████████████████████| 100%


DF.global_pool_output.C1 DF.global_pool_output.C2 DF.global_pool_output.C3 DF.global_pool_output.C4 DF.global_pool_output.C5 DF.global_pool_output.C6 DF.global_pool_output.C7 DF.global_pool_output.C8 DF.global_pool_output.C9 DF.global_pool_output.C10 DF.global_pool_output.C11 DF.global_pool_output.C12 DF.global_pool_output.C13 DF.global_pool_output.C14 DF.global_pool_output.C15 DF.global_pool_output.C16 DF.global_pool_output.C17 DF.global_pool_output.C18 DF.global_pool_output.C19 DF.global_pool_output.C20 DF.global_pool_output.C21 DF.global_pool_output.C22 DF.global_pool_output.C23 DF.global_pool_output.C24 DF.global_pool_output.C25 DF.global_pool_output.C26 DF.global_pool_output.C27 DF.global_pool_output.C28 DF.global_pool_output.C29 DF.global_pool_output.C30 DF.global_pool_output.C31 DF.global_pool_output.C32 DF.global_pool_output.C33 DF.global_pool_output.C34 DF.global_pool_output.C35 DF.global_pool_output.C36 DF.global_pool_output.C37 DF.global_pool_output.C38 DF.global_pool_output.C39 DF.global_pool_output.C40 DF.global_pool_output.C41 DF.global_pool_output.C42 DF.global_pool_output.C43 DF.global_pool_output.C44 DF.global_pool_output.C45 DF.global_pool_output.C46 DF.global_pool_output.C47 DF.global_pool_output.C48 DF.global_pool_output.C49 DF.global_pool_output.C50 DF.global_pool_output.C51 DF.global_pool_output.C52 DF.global_pool_output.C53 DF.global_pool_output.C54 DF.global_pool_output.C55 DF.global_pool_output.C56 DF.global_pool_output.C57 DF.global_pool_output.C58 DF.global_pool_output.C59 DF.global_pool_output.C60 DF.global_pool_output.C61 DF.global_pool_output.C62 DF.global_pool_output.C63 DF.global_pool_output.C64 DF.global_pool_output.C65 DF.global_pool_output.C66 DF.global_pool_output.C67 DF.global_pool_output.C68 DF.global_pool_output.C69 DF.global_pool_output.C70 DF.global_pool_output.C71 DF.global_pool_output.C72 DF.global_pool_output.C73 DF.global_pool_output.C74 DF.global_pool_output.C75 DF.global_pool_output.C76 DF.global_pool_output.C77 DF.global_pool_output.C78 DF.global_pool_output.C79 DF.global_pool_output.C80 DF.global_pool_output.C81 DF.global_pool_output.C82 DF.global_pool_output.C83 DF.global_pool_output.C84 DF.global_pool_output.C85 DF.global_pool_output.C86 DF.global_pool_output.C87 DF.global_pool_output.C88 DF.global_pool_output.C89 DF.global_pool_output.C90 DF.global_pool_output.C91 DF.global_pool_output.C92 DF.global_pool_output.C93 DF.global_pool_output.C94 DF.global_pool_output.C95 DF.global_pool_output.C96 DF.global_pool_output.C97 DF.global_pool_output.C98 DF.global_pool_output.C99 DF.global_pool_output.C100 DF.global_pool_output.C101 DF.global_pool_output.C102 DF.global_pool_output.C103 DF.global_pool_output.C104 DF.global_pool_output.C105 DF.global_pool_output.C106 DF.global_pool_output.C107 DF.global_pool_output.C108 DF.global_pool_output.C109 DF.global_pool_output.C110 DF.global_pool_output.C111 DF.global_pool_output.C112 DF.global_pool_output.C113 DF.global_pool_output.C114 DF.global_pool_output.C115 DF.global_pool_output.C116 DF.global_pool_output.C117 DF.global_pool_output.C118 DF.global_pool_output.C119 DF.global_pool_output.C120 DF.global_pool_output.C121 DF.global_pool_output.C122 DF.global_pool_output.C123 DF.global_pool_output.C124 DF.global_pool_output.C125 DF.global_pool_output.C126 DF.global_pool_output.C127 DF.global_pool_output.C128 DF.global_pool_output.C129 DF.global_pool_output.C130 DF.global_pool_output.C131 DF.global_pool_output.C132 DF.global_pool_output.C133 DF.global_pool_output.C134 DF.global_pool_output.C135 DF.global_pool_output.C136 DF.global_pool_output.C137 DF.global_pool_output.C138 DF.global_pool_output.C139 DF.global_pool_output.C140 DF.global_pool_output.C141 DF.global_pool_output.C142 DF.global_pool_output.C143 DF.global_pool_output.C144 DF.global_pool_output.C145 DF.global_pool_output.C146 DF.global_pool_output.C147 DF.global_pool_output.C148 DF.global_pool_output.C149 DF.global_pool_output.C150 DF.global_pool_output.C151 DF.global_pool_output.C152 DF.g

In [37]:
#print(extracted_features2.ncol)
#assert extracted_features2.ncol == 10976

## Find the squared cosine similarity between the first 10 images and the rest
extracted_features[:10,:].distance(extracted_features[10:,:], "cosine_sq")
#print(df)
#assert df.shape[0] == 257
#assert df.shape[1] == 10


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257
6.07665e-07,5.25738e-07,5.98488e-07,5.38205e-07,6.78461e-07,4.8725e-07,4.90904e-07,6.97056e-07,5.57918e-07,7.13273e-07,5.74591e-07,4.11499e-07,5.607e-07,4.31369e-07,5.6856e-07,4.00154e-07,2.3062e-07,6.2184e-07,4.63244e-07,5.24696e-07,5.63778e-07,3.96504e-07,2.23267e-07,5.57317e-07,6.34519e-07,5.14032e-07,5.68088e-07,3.16806e-07,7.88716e-07,4.08742e-07,4.17128e-07,4.79071e-07,3.31962e-07,5.67385e-07,2.86746e-07,5.38124e-07,3.84958e-07,4.00346e-07,5.70135e-07,5.45811e-07,6.91386e-07,3.18444e-07,2.83977e-07,4.45868e-07,4.85776e-07,4.8835e-07,7.49258e-07,4.86875e-07,6.08341e-07,4.61238e-07,7.76483e-07,4.4593e-07,4.71438e-07,3.59491e-07,5.29198e-07,4.16697e-07,6.52853e-07,7.01552e-07,4.21905e-07,2.79045e-07,5.11652e-07,5.09195e-07,4.25921e-07,3.65599e-07,4.92267e-07,3.12691e-07,5.80962e-07,5.58788e-07,4.34505e-07,7.57199e-07,4.88674e-07,7.02793e-07,6.00381e-07,4.0961e-07,3.65522e-07,8.16693e-07,5.42969e-07,6.62182e-07,3.3741e-07,2.51025e-07,2.72047e-07,2.42642e-07,5.26548e-07,3.37414e-07,4.77018e-07,5.73458e-07,6.11199e-07,4.88483e-07,5.17028e-07,3.39972e-07,3.87609e-07,5.37756e-07,3.9616e-07,6.02669e-07,4.39589e-07,5.3551e-07,3.82015e-07,4.4881e-07,3.9083e-07,5.05798e-07,4.34123e-07,3.03723e-07,5.11504e-07,5.52174e-07,3.48029e-07,4.43086e-07,4.14522e-07,5.84735e-07,5.00356e-07,5.50294e-07,5.01056e-07,3.9412e-07,6.43318e-07,3.94858e-07,3.71134e-07,4.43614e-07,3.9773e-07,4.40981e-07,5.18676e-07,5.75401e-07,3.4343e-07,4.00737e-07,4.98252e-07,6.07412e-07,3.8238e-07,2.40948e-07,2.57494e-07,4.30712e-07,6.66612e-07,5.81565e-07,4.85001e-07,3.96794e-07,6.01248e-07,2.75109e-07,2.3334e-07,4.04255e-07,3.40161e-07,4.35286e-07,4.24329e-07,4.14051e-07,3.87971e-07,4.19245e-07,3.63857e-07,4.60001e-07,4.82936e-07,7.77623e-07,3.48946e-07,5.28394e-07,3.99253e-07,5.51498e-07,4.62323e-07,2.82856e-07,6.15443e-07,4.89857e-07,2.87915e-07,3.71357e-07,3.10314e-07,4.60688e-07,4.29093e-07,5.88465e-07,4.43637e-07,6.46365e-07,5.35839e-07,4.03315e-07,5.03206e-07,5.76616e-07,4.0817e-07,5.17569e-07,4.83035e-07,2.99013e-07,5.18323e-07,3.59522e-07,7.62109e-07,3.79791e-07,3.53583e-07,3.97352e-07,6.44041e-07,6.54292e-07,3.79594e-07,3.74578e-07,3.6207e-07,4.40362e-07,5.71454e-07,5.10869e-07,6.35452e-07,4.68107e-07,3.56275e-07,4.91719e-07,4.09411e-07,3.59085e-07,3.55608e-07,4.85055e-07,3.93384e-07,2.77965e-07,6.03092e-07,5.1775e-07,6.17829e-07,6.88316e-07,4.45196e-07,3.86204e-07,5.81366e-07,5.05111e-07,5.2009e-07,4.08913e-07,4.25588e-07,4.49025e-07,5.29317e-07,5.6309e-07,4.22e-07,4.14782e-07,4.3629e-07,4.07966e-07,3.45201e-07,5.42059e-07,5.40695e-07,5.42824e-07,7.54001e-07,3.9677e-07,4.72749e-07,5.70026e-07,4.12953e-07,5.02988e-07,4.80967e-07,3.4736e-07,4.03725e-07,5.33132e-07,3.23284e-07,3.35718e-07,4.90012e-07,7.1853e-07,5.06097e-07,8.31805e-07,5.6459e-07,2.38923e-07,2.65533e-07,4.5451e-07,3.05862e-07,4.66293e-07,